<a href="https://colab.research.google.com/github/Bergu1/neural-network-project/blob/main/neural_network_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importuje biblioteke

In [1]:
import torch
import kagglehub

In [5]:
import os

In [6]:
path = kagglehub.dataset_download("chrisfilo/fruit-recognition")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/chrisfilo/fruit-recognition/versions/1


In [3]:
path = '/root/.cache/kagglehub/datasets/chrisfilo/fruit-recognition/versions/1'

In [10]:
files = os.listdir(path)
print("Files in the dataset folder:", files)

Files in the dataset folder: ['Apple', 'Mango', 'Pomegranate', 'Kiwi', 'Plum', 'Peach', 'Carambola', 'muskmelon', 'Banana', 'Pear', 'Guava', 'Persimmon', 'Tomatoes', 'Orange', 'Pitaya']
